In [ ]:
pwd

In [ ]:
cd Rar

In [ ]:
cd object-detection-nn

In [ ]:
# cd ../../..

In [ ]:
remote_mode = True

import os
import numpy as np
from nn.YOLO_VGG16_OBB.utils.constants import ANCHORS
from nn.YOLO_VGG16_OBB.prepare_data.dota_dataset_memory import DotaDataset
from nn.YOLO_VGG16_OBB.prepare_data.transforms import train_transform, test_transform
from nn.YOLO_VGG16_OBB.utils.helpers import convert_cells_to_bboxes, load_checkpoint, nms, plot_image, save_checkpoint
from nn.YOLO_VGG16_OBB.utils.constants import device, s, leanring_rate, save_model, checkpoint_file
from nn.YOLO_VGG16_OBB.model.YOLO_VGG16_OBB import YOLO_VGG16_OBB
import torch
import torch.optim as optim
from nn.YOLO_VGG16_OBB.model.loss import YOLOLoss
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
import torchvision.transforms as T

if remote_mode:
    model_path_base = f"/home/dcor/niskhizov/Rar/object-detection-nn/nn/YOLO_VGG16_OBB/notebooks/vgg_f_obb_v2_model"
else:
    model_path_base = f"nn/YOLO_VGG16_OBB/notebooks/"


In [ ]:
categories = ['plane','ship', 'storage-tank', 'baseball-diamond', 'tennis-court', 'basketball-court', 'ground-track-field', 'harbor', 'bridge', 'large-vehicle', 'small-vehicle', 'helicopter', 'roundabout', 'soccer-ball-field', 'swimming-pool']


In [ ]:
# Creating the model from YOLOv3 class 
load_model = True
save_model = False
model = YOLO_VGG16_OBB(num_classes=len(categories)).to(device) 

# Defining the optimizer 
optimizer = optim.Adam(model.parameters(), lr = leanring_rate) 

# Defining the loss function 
loss_fn = YOLOLoss() 

# Defining the scaler for mixed precision training 
scaler = torch.amp.GradScaler(device=device) 
# Loading the checkpoint 
if load_model: 
    load_checkpoint(model_path_base + f"e297_vgg16_{checkpoint_file}", model, optimizer, leanring_rate, device) 

# Initialize TensorBoard writer
# writer = SummaryWriter(log_dir='runs/YOLO_VGG16_OBB_v2')



In [ ]:
dataset = DotaDataset( 
	categories=categories,
	grid_sizes=[13, 26, 52], 
	anchors=ANCHORS, 
	transform=train_transform 
) 

# Defining the train data loader 
train_loader = torch.utils.data.DataLoader( 
	dataset=dataset, 
	batch_size=8, 
	shuffle=True, 
	num_workers=2,
 	prefetch_factor=10,
) 


In [ ]:
val_dataset = DotaDataset(
    categories=categories,
    grid_sizes=[13, 26, 52],
    anchors=ANCHORS,
    transform=test_transform,  # Use the same transform for validation
    data_base_path = f"nn/dotadataset/train"
)

# Create the validation data loader
val_loader = torch.utils.data.DataLoader(
    dataset=val_dataset,
    batch_size=8,
    shuffle=True,
)

val_loader_iter = iter(val_loader)

In [ ]:
# Scaling the anchors 
scaled_anchors = ( 
	torch.tensor(ANCHORS) *
	torch.tensor(s).unsqueeze(1).unsqueeze(1).repeat(1,3,2) 
).to(device) 

In [ ]:
model.eval()

# Getting a sample image from the test data loader 
try:
	x, y = next(val_loader_iter)
except StopIteration:
	val_loader_iter = iter(val_loader)
	x, y = next(val_loader_iter)
x = x.to(device) 

print("###################################### display and report image ######################################")
with torch.no_grad():
	scaled_anchors = ( 
	torch.tensor(ANCHORS) *
	torch.tensor(s).unsqueeze(1).unsqueeze(1).repeat(1,3,2) 
	).to(device) 
	output = model(x)
	y0, y1, y2 = ( 
		y[0].to(device), 
		y[1].to(device), 
		y[2].to(device), 
	) 

	with torch.amp.autocast(device_type=device): 
		# # Getting the model predictions 
		# outputs = model(x) 
		# Calculating the loss at each scale 
		loss = ( 
			loss_fn(output[0], y0, scaled_anchors[0]) 
			+ loss_fn(output[1], y1, scaled_anchors[1]) 
			+ loss_fn(output[2], y2, scaled_anchors[2]) 
		) 

	# TEMP- print target boxes
	bboxes = [[] for _ in range(x.shape[0])]
	for i in range(3):
		batch_size, A, S, _, _ = y[i].shape
		anchor = scaled_anchors[i]
		boxes_scale_i = convert_cells_to_bboxes(y[i], anchor, s=S, is_predictions=False)
		for idx, box in enumerate(boxes_scale_i):
			bboxes[idx] += box

	i = 0
	print('bboxes[i] shape:', np.array(bboxes[i]).shape)
	nms_boxes = nms(bboxes[i], iou_threshold=0.6, threshold=0.4)
	img_with_boxes = plot_image(x[i].permute(1, 2, 0).detach().cpu(), nms_boxes, categories)
	img_with_boxes = T.ToTensor()(img_with_boxes)

	# # Print predictions
	# writer.add_scalar('Loss/val', loss.item(), e * len(train_loader) + batch_idx)

	bboxes = [[] for _ in range(x.shape[0])]
	for i in range(3):
		batch_size, A, S, _, _ = output[i].shape
		anchor = scaled_anchors[i]
		boxes_scale_i = convert_cells_to_bboxes(output[i], anchor, s=S, is_predictions=True)
		for idx, box in enumerate(boxes_scale_i):
			bboxes[idx] += box

	i = 0
	print('bboxes[i] shape:', np.array(bboxes[i]).shape)
	nms_boxes = nms(bboxes[i], iou_threshold=1, threshold=0.1)
	img_with_boxes = plot_image(x[i].permute(1, 2, 0).detach().cpu(), nms_boxes, categories)
	img_with_boxes = T.ToTensor()(img_with_boxes)
	# writer.add_image(f'Val/Image_{e}_{i}_{batch_idx}_before', img_with_boxes, e * len(train_loader) + batch_idx)

# model.train()
# except Exception as error:
# 	print(error)
# 	error_counter += 1
# 	if error_counter > 10:
# 		raise error


#################
# training_loop(train_loader, model, optimizer, loss_fn, scaler, scaled_anchors) 


In [ ]:
pred = output[1]
target = y1 
anchors = scaled_anchors[1]
pred_angle = torch.tanh(pred[..., 5])

In [ ]:
pred[..., 1:3].shape, pred[..., 3:5].shape, pred_angle.unsqueeze(-1).shape, anchors.shape

In [ ]:
torch.exp(pred[..., 3:5]) * anchors